# Deepersense: sim data generator

## imports

In [1]:
import os
# import sys
import rospy
import rosbag
import rospkg
import glob

In [2]:
from simulation_controller import SimulationController
from scene_generator import SceneGenerator, ColorMap
from launchfile_maker import LaunchFileMaker

## file manager

### cwd

In [3]:
cwd = os.getcwd()

### package path

In [4]:
rospack = rospkg.RosPack()
package_path = rospack.get_path('stonefish_scene_generator')

### iterate files in folder

In [5]:
all_files_path = os.path.join(package_path,'*')
for file_name in glob.glob(all_files_path):
    print(file_name)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch
/home/zeged/catkin_ws/src/stonefish_scene_generator/src
/home/zeged/catkin_ws/src/stonefish_scene_generator/package.xml
/home/zeged/catkin_ws/src/stonefish_scene_generator/data
/home/zeged/catkin_ws/src/stonefish_scene_generator/CMakeLists.txt


### create data folder

In [6]:
data_folder_path = os.path.join(package_path, 'data')
dir_exists = os.path.isdir(data_folder_path)
if not dir_exists:
    os.mkdir(data_folder_path)
    print('data folder created: {}'.format(data_folder_path))

## generate stonefish scene

### generate scene

In [7]:
title = 'basic'
seabed_depth = 15
gen = SceneGenerator(seabed_depth)
gen.add_empty_frame(ColorMap.jet)
gen.add_reefs()
scene = gen.generate()
# print(scene)

### save scene to .scn file

In [8]:
scene_file_path = os.path.join(data_folder_path, title + '.scn')
f = open(scene_file_path, "w")
f.write(scene)
f.close()

## Simulation

### get launch_file_path

In [9]:
launch_file_name = 'auto_generated.launch'
launch_file_path = os.path.join(package_path, 'launch', launch_file_name)
print(launch_file_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch/auto_generated.launch


### edit/make launch file

In [10]:
rate = 300.0
resolution = (1200, 800)
output_bagfile_path = os.path.join(data_folder_path, title + '.bag')
maker = LaunchFileMaker(scene_file_path, rate, resolution, output_bagfile_path)
maker.save_to_file(launch_file_path)

### setup simulation

In [11]:
controller = SimulationController(launch_file_path)

... logging to /home/zeged/.ros/log/0f6a1c1a-289e-11ec-ab87-107b448dffed/roslaunch-zeged-System-Product-Name-4341.log


### launch simulation

In [12]:
# controller.run()
# controller.wait_for_ros()

## Bag

### open bag file

In [13]:
import rospy
from collections import defaultdict
import cv2
import numpy as np
from cv_bridge import CvBridge

bag = rosbag.Bag(bagfile, 'r')

In [14]:
bagfiles = glob.glob(os.path.join(data_folder_path, '*.bag'))
print(bagfiles)
bagfile_path = bagfiles[0]
bag = rosbag.Bag(bagfile_path, 'r')

['/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic_2021-10-09-03-15-17.bag', '/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic_2021-10-09-03-11-50.bag']


In [15]:
def imshow(win, img):
    cv2.imshow(win, img)
    cv2.waitKey(10)
     

In [16]:
def process_image(msg, topic):
    bridge = CvBridge()
    encoding = 'bgr8'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [17]:
def process_depth(msg, topic):
    bridge = CvBridge()
    encoding = '32FC1'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [18]:
def normaliaze_depth(msg):
    cv_image_array = np.array(msg, dtype = np.dtype('f8'))
    slice1Copy = cv_image_norm = cv2.normalize(cv_image_array, cv_image_array, 0, 1, cv2.NORM_MINMAX)
    slice1Copy = np.uint8(slice1Copy*255)
    return(slice1Copy)

In [19]:
iterator = bag.read_messages()
counter = defaultdict(lambda: 0)
for topic, msg, time in iterator:
    msg_time = rospy.Time.to_sec(time) # .to_time(time)
    msg_type = str(type(msg)).split('__')[1].split("'")[0]
    print(msg_time, msg_type, topic)
    counter[topic]+=1

    if 'image_color' in topic or 'display' in topic:
        bridged_image = process_image(msg, str(topic))
        imshow(topic, bridged_image)


    if 'image_depth2/image_depth' in topic:
        bridged_image = process_depth(msg, str(topic))
        norm_image = normaliaze_depth(bridged_image)
        imshow(topic, norm_image)

cv2.destroyAllWindows()

(1633738530.8430107, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633738530.87811, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633738530.879607, 'Image', '/camera2/image_raw/image_color')
(1633738530.9009678, 'CameraInfo', '/image_depth2/camera_info')
(1633738530.9031887, 'Image', '/image_depth2/image_depth')


NameError: global name 'img' is not defined

In [ ]:
print(counter)

defaultdict(<function <lambda> at 0x7f006c9911d0>, {'/sparus2/Blue_view_M900_FLS2/display': 176, '/camera2/image_raw/image_color': 252, '/image_depth2/camera_info': 175, '/camera2/image_raw/camera_info': 252, '/image_depth2/image_depth': 175})
